# Provisioning OpenRouter API keys

This notebook is for instructors to provision OpenRouter API keys for workshop participants. It is not intended for use by participants.

However intermediate and advanced participants are welcome to see the code and use it as a reference whenever they need to bulk provision API keys for their own projects.

## OpenRouter API provisioning

OpenRouter provides a simple API to create and manage API keys. 

The core idea is to use a special API key that has the `admin` role, which allows you to create new API keys for other users. This key does not make regular API calls but is used to manage API keys.

This type of key is extremely powerful and should be kept secret. It is not intended for use in production applications or by end users.

If this type of key is compromised, it can be used to create new API keys for any user, which can then be used to make API calls and drain your account's balance to zero.

In [22]:
# first let's load needed libraries and also read system environment variables
# let's load Python sys to see what Python version we are using
import sys
print(f"Python version: {sys.version}")
# now datetime
from datetime import datetime
print(f"Current date and time: {datetime.now()}")
# we will need json to read and write JSON files
import json
# and os to read environment variables
import os

# we also need Pathlib to work with file paths
from pathlib import Path

# now let's read environment variables

# OPEN_ROUTER_BSSDH_PROVISIONER
open_router_bssdh_provisioner = os.getenv("OPEN_ROUTER_BSSDH_PROVISIONER")
if open_router_bssdh_provisioner:
    print("OPEN_ROUTER_BSSDH_PROVISIONER key loaded from environment variables.")
    # README! we do not want to print the key itself for security reasons it is very easy to print it and publish it by mistake
else:
    print("OPEN_ROUTER_BSSDH_PROVISIONER key not found in environment variables. Please set it before running the script.")


# we will need some external libraries, so let's check if they are installed
# first requests
try:
    import requests
    print(f"requests library is installed: {requests.__version__}")
except ImportError:
    print("requests library is not installed. Please install it using 'pip install requests' from command line terminal.")

Python version: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Current date and time: 2025-07-25 12:33:54.132865
OPEN_ROUTER_BSSDH_PROVISIONER key loaded from environment variables.
requests library is installed: 2.32.4


## Provisioning API keys

### Creating function to create API keys

First we need to create a function that will use the OpenRouter API to create new API keys. This function name, label and limit with some defaults that can be overridden by the caller.


In [ ]:
# now that everything is loaded, let's define a function to provision API keys
def provision_open_router_keys(provision_key=open_router_bssdh_provisioner, 
                               name="Test Customer Instance Key",
                               label="TestCustomer123",
                               limit=1): # 1 USD dollar limit by default
    # we use example from https://openrouter.ai/docs/features/provisioning-api-keys
    PROVISIONING_API_KEY = provision_key
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    # Create a new API key
    response = requests.post(
        # f"{BASE_URL}/", # this is WRONG! no need for trailing slash
        url = BASE_URL,
        headers={
            "Authorization": f"Bearer {PROVISIONING_API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "name": name,   # Name of the API key
            "label": label,
            "limit": limit  # Optional credit limit
        }
    )
    # return the response
    if response.status_code == 201:
        print("API key created successfully.")
        return response.json()  # Return the JSON response containing the API key details
    else:
        print(f"Failed to create API key: {response.status_code} - {response.text}")
        return None
    
# let's test the function to create a new API key
new_key = provision_open_router_keys(
    name="BSSDH 2025 Workshop",
    label="BSSDH 2025 Workshop Key",
    limit=2
)

API key created successfully.


In [21]:
# let's try creating a budget key with 0.50 usd limit
new_budget_key = provision_open_router_keys(
    name="BSSDH 2025 Workshop Budget Key",
    label="BSSDH 2025 Workshop Budget Key",
    limit=0.50
)


API key created successfully.


In [24]:
# let's save keys to individual files in temp folder that is sibling the notebook folder
save_path = Path("../temp")
# note: I have excluded temp folder by adding temp entry in .gitignore file so it will not be committed to the repository
# otherwise everyone in the world would have access to your API keys!!!! if your repository is public
# create temp folder if it does not exist
save_path.mkdir(parents=True, exist_ok=True)
# save new_key to a file
if new_key:
    with open(save_path / "new_key.json", "w") as f:
        json.dump(new_key, f, indent=4)
    print(f"New key saved to {save_path / 'new_key.json'}")
# save new_budget_key to a file
if new_budget_key:
    with open(save_path / "new_budget_key.json", "w") as f:
        json.dump(new_budget_key, f, indent=4)
    print(f"New budget key saved to {save_path / 'new_budget_key.json'}")

New key saved to ..\temp\new_key.json
New budget key saved to ..\temp\new_budget_key.json


## Getting Previously created keys
We can use the OpenRouter API to get a list of previously created API keys. This is useful to check if the keys were created successfully and to see the limits and other details of the keys.

Again be very careful with retrieved keys, as they can be used to make API calls and drain your account's balance.

In [9]:
def get_keys(provision_key=open_router_bssdh_provisioner):
    # we use example from https://openrouter.ai/docs/features/provisioning-api-keys
    PROVISIONING_API_KEY = provision_key
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    # List the most recent 100 API keys
    response = requests.get(
        BASE_URL,
        headers={
            "Authorization": f"Bearer {PROVISIONING_API_KEY}",
            "Content-Type": "application/json"
        }
    )

    # return the response
    if response.status_code == 200:
        print("API keys retrieved successfully.")
        return response.json()  # Return the JSON response containing the API keys
    else:
        print(f"Failed to retrieve API keys: {response.status_code} - {response.text}")
        return None
    
keys = get_keys()
# how many keys we have?
if keys:
    print(f"Number of keys retrieved: {len(keys)}")


API keys retrieved successfully.
Number of keys retrieved: 1


In [16]:
# print keys.keys()
# keys is a dictionary so it has keys ...
print("Keys available in the response:")
for key in keys.keys():
    print(f"key - {key}")
# data actually holds the list of keys but we will not print it here as it may contain sensitive information
# we will just see how many keys we have
if 'data' in keys:
    print(f"Number of API keys in data: {len(keys['data'])}")

Keys available in the response:
key - data
Number of API keys in data: 5


In [26]:
# let's get first key that contains Segmentation in its name
# in our particular case this key is already used so we are not afraid to print it by accident but generally be very careful with retrieved keys, 
# theoretically hash of the key can be used to identify the key without revealing it
if keys and 'data' in keys:
    segmentation_keys = [key for key in keys['data'] if 'Segmentation' in key['name']]
    if segmentation_keys:
        print(f"Found {len(segmentation_keys)} keys with 'Segmentation' in their name.")
        # print("First key with 'Segmentation':", segmentation_keys[0]) # prints hash only but still be careful with this
    else:
        print("No keys found with 'Segmentation' in their name.")

Found 1 keys with 'Segmentation' in their name.
